# Recipe

This notbook is used to defining the methodology for identifying important features in wine quality datasets.

In [1]:
%%capture
%run 06_model_building.ipynb

## 1. Identify Important Features

In [2]:
def identyfy_three_important_features(evaluation_matrix_list): 
    '''
    This function identifies the three most important features from each evaluation matrix (which could be model results or analysis outputs) in the given list. 
    It is used to pinpoint key factors influencing wine quality.
    '''
    result_dict = []
    # Iterate through unique dataframes
    for dataframe in evaluation_matrix_list:
        df_subset = dataframe[0].iloc[1:].copy()
        df_subset['Coefficients'] = df_subset['Coefficients'].abs()
        
        # Sort the subset based on coefficients in descending order
        sorted_subset = df_subset.sort_values(by='Coefficients', ascending=False).reset_index(drop=True)

        # Select the top three important features
        top_three_features = sorted_subset['Parameters'].loc[:2:].tolist()   
        # Create a dictionary entry for the current dataframe
        results_df = result_dict.append({dataframe[0]['df'][1] : top_three_features})
        
    return result_dict

## 2. Finding the receipe for the good and poor wine

This section is dedicated to applying the methodologies or processes established earlier to specifically categorize and analyze good and poor quality wines.

In [3]:
# Here, the function to identify top three important features is called with the evaluation matrix list as the argument. 
# The output will be used to understand critical factors in wine quality.
top_three_features_list = identyfy_three_important_features(evaluation_matrix_list=evaluation_matrix_list)

NameError: name 'evaluation_matrix_list' is not defined

In [ ]:
features_to_reverse_white = ['chlorides', 'volatile acidity']
features_to_reverse_red = ['residual sugar', 'chlorides']

def correct_reverse_log_transformation(df, features):
    '''
    This function reverses the logarithmic transformation applied to certain features in the dataframe. 
    This is useful for converting transformed data back to its original scale, often for interpretation or further analysis.    
    '''
    for feature in features:
        if feature in df.columns:
            df[feature] = np.exp(df[feature])
    return df

df_white_good_without_outliers = correct_reverse_log_transformation(df_white_good_without_outliers, features_to_reverse_white)
df_white_poor_without_outliers = correct_reverse_log_transformation(df_white_poor_without_outliers, features_to_reverse_white)

df_red_good_without_outliers = correct_reverse_log_transformation(df_red_good_without_outliers, features_to_reverse_red)
df_red_poor_without_outliers = correct_reverse_log_transformation(df_red_poor_without_outliers, features_to_reverse_red)

In [ ]:
def get_receipes(top_three_features_list, df_lists):
    '''
    his function calculates and prints the 'recipes' for different types of wine, based on the top three features identified earlier. 
    It computes the mean and standard deviation for these features, using these to define a range that characterizes each wine type.
    '''
    for df in df_lists:
        pipeline_name = f"Pipeline_{get_wine_str(df)}"
        for model in top_three_features_list:
            if pipeline_name in model:
                feature_list = model[pipeline_name]
                numeric_columns = check_numeric_columns(df)
                numeric_df = df[numeric_columns]

                means = numeric_df[feature_list].mean()
                stds = numeric_df[feature_list].std()

                ranges = {feature: (means[feature] - stds[feature], means[feature] + stds[feature]) for feature in feature_list}

                print(f"\nRecipes of {get_wine_str(df)} wine:")
                for feature, range_vals in ranges.items():
                    print(f"{feature.capitalize()}: {range_vals[0]:.2f} - {range_vals[1]:.2f}")


In [ ]:
get_receipes(top_three_features_list, wine_quality_without_outliers_dfs) 